<a href="https://colab.research.google.com/github/sainijagjit/CS6910-A2/blob/main/Part%20A/Assignment_2_iNaturalist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install pytorch-lightning --quiet
# install weights and biases
# !pip uninstall wandb -y
!pip install wandb --quiet
# !pip install --upgrade wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as L
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler, Subset
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import EarlyStopping,Callback,ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import wandb
from google.colab import userdata

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
zip_path = "/content/drive/MyDrive/Deep Learning/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip

In [6]:
wandb.login(key=userdata.get('wandb_key'))
wandb_project="CS6910 - Assignment 2"
wandb_entity="ns24z060"

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


**Loading & Splitting Dataset**

In [18]:
class DataModule(L.LightningDataModule):
    def __init__(self, batch_size=64, augment_data= True ,data_dir = './inaturalist_12K',num_workers=2):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.augment_data = augment_data
        self.num_workers=num_workers


        self.augmentation = transforms.Compose([
              transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
              transforms.RandomRotation(degrees=15),
              transforms.RandomHorizontalFlip(),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        self.transform = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])



    def prepare_data(self):
        self.train_dataset = ImageFolder(root=f'{self.data_dir}/train')
        self.test_dataset = ImageFolder(root=f'{self.data_dir}/val')
        print(f'Classes {self.train_dataset.classes}')
        print(f'Test Samples {len(self.test_dataset)}')


    def setup(self, stage=None):
        self.test_dataset.transform = self.transform
        labels = self.train_dataset.targets
        train_indices, val_indices = train_test_split(
            range(len(self.train_dataset)),
            test_size=0.2,
            random_state=42,
            stratify=labels
        )
        if stage=='fit':
          class_names = self.train_dataset.classes
          class_counts = {class_name: 0 for class_name in class_names}
          for val_indice in val_indices:
              _, class_idx = self.train_dataset[val_indice]
              class_name = class_names[class_idx]
              class_counts[class_name] += 1
          print('Validation Datased Class Distribution')
          for class_name, count in class_counts.items():
              print(f"Class: {class_name}, No. of samples: {count}")
        self.val_dataset = Subset(self.train_dataset, val_indices)
        self.train_dataset = Subset(self.train_dataset, train_indices)
        if stage=='fit':
          print(f'Train Samples {len(self.train_dataset)}')
          print(f'Val Samples {len(self.val_dataset)}')

        if self.augment_data:
          self.train_dataset.dataset.transform = self.augmentation
        self.val_dataset.dataset.transform = self.transform

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)


    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)


    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.num_workers)



In [12]:
class Net(nn.Module):
  def __init__(self,
                activation_function,
                num_filters,
                dense_neurons,
                batch_size,
                batch_norm,
                dropout,
                filter_size=3,
                stride=2
                ):
      super().__init__()
      self.batch_size=batch_size
      cnn_layers = []
      for i in range(5):
        in_channels = 3 if i==0 else num_filters[i-1]
        cnn_layers.extend([nn.Conv2d(in_channels, num_filters[i], kernel_size=filter_size),
                      getattr(nn, activation_function)(),
                      ])
        if batch_norm:
          cnn_layers.append(nn.BatchNorm2d(num_filters[i],affine=True))
        cnn_layers.append(nn.MaxPool2d(kernel_size=filter_size,stride=stride))

      self.cnn_net = nn.Sequential(*cnn_layers)
      n_sizes = self._get_conv_output((3,224,224))
      dnn_layers = [
          nn.Flatten(),
          nn.Linear(n_sizes, dense_neurons)]
      if dropout:
        dnn_layers.append(nn.Dropout(dropout))
      dnn_layers.extend([
          getattr(nn, activation_function)(),
          nn.Linear(dense_neurons, 10)])
      self.dnn_net = nn.Sequential(*dnn_layers)

  def _get_conv_output(self, shape):
        input = torch.autograd.Variable(torch.rand(self.batch_size, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(self.batch_size, -1).size(1)
        return n_size

  def _forward_features(self, x):
        return self.cnn_net(x)

  def forward(self, x):
        x =  self.cnn_net(x)
        x =  self.dnn_net(x)
        return x

In [13]:
class MyCnnModel(L.LightningModule):
    def __init__(self,
                 activation_function = 'ReLU',
                 num_filters=[32,32,32,32,32],
                 dense_neurons=1024,
                 learning_rate=0.01,
                 batch_size=64,
                 dropout=0.5,
                 batch_norm=True,
                 loss_function='cross_entropy',
                 optimizer='Adam'):
        super().__init__()
        self.save_hyperparameters()
        self.model = Net(activation_function,num_filters,dense_neurons,batch_size,batch_norm,dropout)
        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.loss_function = getattr(F, loss_function)
        self.learning_rate = learning_rate
        self.optimizer = optimizer

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
      loss,acc = self._common_step(batch,batch_idx)
      self.log('train_loss', loss, on_step=False, on_epoch=True, logger=True)
      self.log('train_acc', acc, on_step=False, on_epoch=True, logger=True)
      return loss

    def validation_step(self, batch, batch_idx):
      loss,acc =  self._common_step(batch,batch_idx)
      self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      return loss


    def test_step(self,batch,batch_idx):
      loss,acc =  self._common_step(batch,batch_idx)
      self.log('test_loss', loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      self.log('test_acc', acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      return loss

    def _common_step(self, batch, batch_idx):
      inputs, target = batch
      output = self.forward(inputs)
      loss = self.loss_function(output, target)
      acc = self.accuracy(output, target)
      return loss,acc

    def configure_optimizers(self):
      return getattr(torch.optim,self.optimizer)(self.parameters(), lr=self.learning_rate)

In [27]:
class ImagePredictionLogger(L.callbacks.Callback):
    def __init__(self, samples, num_samples=30):
        super().__init__()
        self.num_samples = num_samples
        self.sample_imgs, self.val_labels = samples
        self.classes = classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

    def on_validation_epoch_end(self, trainer, pl_module):
        sample_imgs = self.sample_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        logits = pl_module(sample_imgs)
        preds = torch.argmax(logits, -1)

        trainer.logger.experiment.log({
            "examples":[wandb.Image(x, caption=f"Pred:{self.classes[pred]}, Label:{self.classes[y]}")
                           for x, pred, y in zip(sample_imgs[:self.num_samples],
                                                 preds[:self.num_samples],
                                                 val_labels[:self.num_samples])]
            })

    def on_test_epoch_end(self, trainer, pl_module):
        sample_imgs = self.sample_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        logits = pl_module(sample_imgs)
        preds = torch.argmax(logits, -1)

        trainer.logger.experiment.log({
            "Test Predictions":[wandb.Image(x, caption=f"Pred:{self.classes[pred]}, Label:{self.classes[y]}")
                           for x, pred, y in zip(sample_imgs[:self.num_samples],
                                                 preds[:self.num_samples],
                                                 val_labels[:self.num_samples])]
            })


In [16]:
def train():

    config_defaults = {
        "num_filters": [32,32,32,32,32],
        "batch_norm": True,
        "augment_data": True,
        "dropout":0.5,
        "learning_rate": 1e-3,
        "activation_function": 'ReLU',
        "batch_size": 32,
        "epochs": 10,
        "dense_neurons": 512
    }

    wandb.init(config=config_defaults, magic=True)
    config = wandb.config

    name = "_".join(["n_filter", str(config.num_filters), "lr", str(config.learning_rate), "augmt", str(config.augment_data),
                      "drop", str(config.dropout), "b_nrm", str(config.batch_norm),"dse_nrn",str(config.dense_neurons),"batch",str(config.batch_size),"activ",config.activation_function,"eph",str(config.epochs)])
    api = wandb.Api()
    runs = api.runs(wandb.run.project, {"$and": [{"config.name": name}]})
    if len(runs) > 0:
        return
    wandb.run.name = name
    dm = DataModule(config.batch_size,config.augment_data)
    dm.prepare_data()
    dm.setup()
    val_samples = next(iter(dm.val_dataloader()))
    model = MyCnnModel(config.activation_function,config.num_filters,config.dense_neurons,config.learning_rate,config.batch_size,config.dropout,config.batch_norm)
    checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode='max',dirpath='/content/drive/MyDrive/Deep Learning',save_top_k=1,filename='best_model')
    wandb_logger = WandbLogger(project=wandb_project, job_type='train', log_model='all')
    early_stop_callback = EarlyStopping(monitor="val_loss",patience=3)
    trainer = L.Trainer(
      max_epochs=config.epochs,
      check_val_every_n_epoch=1,
      log_every_n_steps=1,
      logger=wandb_logger,
      # fast_dev_run=1,
      # limit_train_batches=0.01,
      # limit_val_batches=0.03125,
      callbacks=[checkpoint_callback,ImagePredictionLogger(val_samples),early_stop_callback])
    trainer.fit(model, dm)
    best_model_path = checkpoint_callback.best_model_path
    wandb.log({f'best_model_path{wandb.run.name}': best_model_path})

In [12]:
sweep_config = {
  "name": "Assignment_2_iNaturalis",
  "method": "bayes",
  'metric': {
      'name': 'val_acc',
      'goal': 'maximize'
    },
  "parameters": {
        "num_filters": {
            "values": [[32,32,32,32,32],[16,32,64,128,256],[256,128,64,32,16],[128,128,64,64,32]]
        },
        "batch_norm":{
          "values": [True,False]
        },
        "augment_data":{
          "values": [True,False]
        },
        "dropout":{
            "values": [0,0.2,0.3,0.5]
        },
        "learning_rate": {
            "values": [1e-3,1e-4,1e-5]
        },
        "activation_function": {
            "values": [ "ReLU","GELU","SELU"]
        },
        "batch_size": {
            "values": [8,32,64,128]
        },
        "epochs": {
            "values": [10,15,20]
        },
        "dense_neurons": {
            "values": [128,512,1024]
        }
    }
}


# sweep_id = wandb.sweep(sweep_config, entity=wandb_entity, project=wandb_project)
wandb.agent("82wf4qdi", train, count=50, entity=wandb_entity, project=wandb_project)
wandb.finish()

wandb: Agent Starting Run: 5r0z9fdv with config:
wandb: 	activation_function: GELU
wandb: 	augment_data: False
wandb: 	batch_norm: False
wandb: 	batch_size: 32
wandb: 	dense_neurons: 128
wandb: 	dropout: 0
wandb: 	epochs: 15
wandb: 	learning_rate: 0.0001
wandb: 	num_filters: [32, 32, 32, 32, 32]
wandb: Currently logged in as: ns24z060. Use `wandb login --relogin` to force relogin
wandb: WARNING wandb.init() arguments ignored because wandb magic has already been initialized


wandb: Ctrl + C detected. Stopping sweep.


wandb: ERROR Control-C detected -- Run data was not synced
/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py:2265: UserWarning: Run (5r0z9fdv) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),


Load Best Model & Check Test Accuracy

In [29]:
model = MyCnnModel.load_from_checkpoint(checkpoint_path='/content/drive/MyDrive/Deep Learning/best_model-v5.ckpt')
dm = DataModule()
dm.prepare_data()
dm.setup()
test_samples = next(iter(dm.test_dataloader()))
wandb_logger = WandbLogger(project=wandb_project, job_type='test', name='test_images_prediction')
trainer = L.Trainer(logger=wandb_logger,callbacks=[ImagePredictionLogger(test_samples)])
trainer.test(model,dm)
wandb.finish()

Classes ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Test Samples 2000


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Classes ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Test Samples 2000


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.30550000071525574    │
│         test_loss         │    2.0237512588500977     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.0237512588500977, 'test_acc': 0.30550000071525574}]